<a href="https://colab.research.google.com/github/adarshtomar333/Hand-Written-Character-Recognition/blob/main/hcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os

In [ ]:
os.environ['KAGGLE_CONFIG_DIR']  = '/content'
!kaggle datasets download -d vaibhao/handwritten-characters
!unzip \*.zip && rm *.zip

In [ ]:
# Verify if the directories exist and print some sample files
train_image_dir = '/content/Train'
test_image_dir = '/content/Validation'

if not os.path.exists(train_image_dir) or not os.path.exists(test_image_dir):
    raise Exception("Image directories not found. Please check the paths.")

# print("Sample training images:", os.listdir(train_image_dir)[:5])
# print("Sample testing images:", os.listdir(test_image_dir)[:5])

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

# Define the paths to the directories
train_dir = '/content/dataset/Train'
test_dir = '/content/dataset/Validation'

# Load and preprocess the training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True
)

# Load and preprocess the testing dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',
    batch_size=32,
    image_size=(28, 28),
    shuffle=True
)

# Extract class names before mapping
class_names = train_dataset.class_names
num_classes = len(class_names)

# Normalize the images to the range [0, 1]
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_dataset = train_dataset.map(normalize_img)
test_dataset = test_dataset.map(normalize_img)

# Convert datasets to use in the model
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(f'Test accuracy: {test_acc}')


In [ ]:
# Visualize training results
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

In [ ]:
# Save the model to an .h5 file
model.save('handwritten_character_recognition_model.h5')

# Save the model architecture to a JSON file
model_json = model.to_json()
with open("model_architecture.json", "w") as json_file:
    json_file.write(model_json)

# Save the model weights to an HDF5 file
model.save_weights("model_weights.h5")

In [ ]:
# Import the files module for downloading files
from google.colab import files

# Download the .h5 file
files.download('handwritten_character_recognition_model.h5')

# Download the JSON file
files.download('model_architecture.json')

# Download the weights file
files.download('model_weights.h5')